In [21]:
#uncomment and run the below line to install geocoder
#%conda install --name metis -c conda-forge geocoder

In [22]:
import pandas as pd
import pickle
import geocoder

In [23]:
with open('pickle/station_lookup.pickle', 'rb') as to_read: #run stop_id_matchup to generate
    stations = pickle.load(to_read)

In [24]:
stations.head()

,C/A,STATION,stop_id,Stop Name,GTFS Latitude,GTFS Longitude,Borough,Daytime Routes
0,A002,59 ST,R11,Lexington Av/59 St,40.762660,-73.967258,M,N W R
1,A006,5 AV/59 ST,R13,5 Av/59 St,40.764811,-73.973347,M,N W R
2,A007,5 AV/59 ST,R13,5 Av/59 St,40.764811,-73.973347,M,N W R
3,A010,57 ST-7 AV,R14,57 St - 7 Av,40.764664,-73.980658,M,N Q R W
4,A011,57 ST-7 AV,R14,57 St - 7 Av,40.764664,-73.980658,M,N Q R W


In [25]:
#we try to use a pickle here to prevent api pulls
try:
    with open('pickle/station_zip.pickle', 'rb') as to_read:
        station_zip = pickle.load(to_read)
except FileNotFoundError:
    station_zip = stations.copy(deep=True)
    station_zip['Zipcodes'] = station_zip.apply(lambda row: geocoder.google([row['GTFS Latitude'], 
                                                                  row['GTFS Longitude']], 
                                                                  method = 'reverse', 
                                                                  key ='YOUR KEY HERE')
                                                                  .postal, axis=1)
    
    with open('pickle/station_zip.pickle', 'wb') as to_write:
        pickle.dump(station_zip, to_write)

In [26]:
stations = pd.merge(stations,station_zip[['STATION','Zipcodes']],on='STATION',how='left')
stations.head()

,C/A,STATION,stop_id,Stop Name,GTFS Latitude,GTFS Longitude,Borough,Daytime Routes,Zipcodes
0,A002,59 ST,R11,Lexington Av/59 St,40.76266,-73.967258,M,N W R,10022
1,A002,59 ST,R11,Lexington Av/59 St,40.76266,-73.967258,M,N W R,11220
2,A002,59 ST,R11,Lexington Av/59 St,40.76266,-73.967258,M,N W R,11220
3,A002,59 ST,R11,Lexington Av/59 St,40.76266,-73.967258,M,N W R,10022
4,A002,59 ST,R11,Lexington Av/59 St,40.76266,-73.967258,M,N W R,10022


In [27]:
with open('pickle/census_clean.pickle', 'rb') as to_read: #run census_data to generate
    census = pickle.load(to_read)

In [28]:
stations.dtypes

C/A                object
STATION            object
stop_id            object
Stop Name          object
GTFS Latitude     float64
GTFS Longitude    float64
Borough            object
Daytime Routes     object
Zipcodes           object
dtype: object

In [29]:
census.dtypes

ZIP                   object
Percent_Female       float64
Percent_White        float64
Percent_Non_White    float64
dtype: object

In [30]:
census = census.rename(columns={'ZIP':'Zipcodes'})

In [31]:
census.head()

,Zipcodes,Percent_Female,Percent_White,Percent_Non_White
0,06390,0.500,0.962,-95.2
1,10001,0.517,0.650,-64.0
2,10002,0.526,0.313,-30.3
3,10003,0.523,0.764,-75.4
4,10004,0.479,0.702,-69.2


In [32]:
print(len(stations))

2426


In [33]:
stations = stations.merge(census, how= 'left',on ='Zipcodes')

In [34]:
print(len(stations))

2426


In [35]:
stations.head()

,C/A,STATION,stop_id,Stop Name,GTFS Latitude,GTFS Longitude,Borough,Daytime Routes,Zipcodes,Percent_Female,Percent_White,Percent_Non_White
0,A002,59 ST,R11,Lexington Av/59 St,40.76266,-73.967258,M,N W R,10022,0.546,0.864,-85.4
1,A002,59 ST,R11,Lexington Av/59 St,40.76266,-73.967258,M,N W R,11220,0.495,0.306,-29.6
2,A002,59 ST,R11,Lexington Av/59 St,40.76266,-73.967258,M,N W R,11220,0.495,0.306,-29.6
3,A002,59 ST,R11,Lexington Av/59 St,40.76266,-73.967258,M,N W R,10022,0.546,0.864,-85.4
4,A002,59 ST,R11,Lexington Av/59 St,40.76266,-73.967258,M,N W R,10022,0.546,0.864,-85.4


In [36]:
with open('pickle/mta_data_daily.pickle', 'rb') as to_read: #run scrape_clean to generate
    mta_data_daily_w = pickle.load(to_read)

In [37]:
mta_data_daily_w.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE,TIME_DELTA,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA
0,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0
1,A002,R051,02-00-00,59 ST,NQR456W,2019-03-31,1 days,571.0,228.0,799.0
2,A002,R051,02-00-00,59 ST,NQR456W,2019-04-02,1 days,1593.0,554.0,2147.0
3,A002,R051,02-00-00,59 ST,NQR456W,2019-04-03,1 days,1652.0,424.0,2076.0
4,A002,R051,02-00-00,59 ST,NQR456W,2019-04-04,1 days,1638.0,511.0,2149.0


In [38]:
print(len(mta_data_daily_w))

212252


In [39]:
mta_data_daily_w = mta_data_daily_w.merge(stations[['C/A','Zipcodes', 'Percent_Female','Percent_Non_White']],
                                      how='left', on = 'C/A')

In [40]:
mta_data_daily_w.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE,TIME_DELTA,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA,Zipcodes,Percent_Female,Percent_Non_White
0,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,10022,0.546,-85.4
1,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,11220,0.495,-29.6
2,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,11220,0.495,-29.6
3,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,10022,0.546,-85.4
4,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,10022,0.546,-85.4


In [41]:
mta_data_daily_w['Weighted_Traffic'] = mta_data_daily_w['TOTAL_DELTA'].multiply(mta_data_daily_w['Percent_Female'], axis = 'index') 

In [42]:
mta_data_daily_w.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATE,TIME_DELTA,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA,Zipcodes,Percent_Female,Percent_Non_White,Weighted_Traffic
0,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,10022,0.546,-85.4,650.832
1,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,11220,0.495,-29.6,590.040
2,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,11220,0.495,-29.6,590.040
3,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,10022,0.546,-85.4,650.832
4,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,1 days,893.0,299.0,1192.0,10022,0.546,-85.4,650.832


In [43]:
print(len(mta_data_daily_w))

730919


In [44]:
with open('pickle/mta_data_intra.pickle', 'rb') as to_read: #run scrape_clean to generate
    mta_data_intra_w = pickle.load(to_read)

In [45]:
print(len(mta_data_intra_w))

1443776


In [46]:
mta_data_intra_w = mta_data_intra_w.merge(stations[['C/A','Zipcodes', 'Percent_Female','Percent_Non_White']],
                                      how='left', on = 'C/A')

In [47]:
mta_data_intra_w.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATETIME,DATE,TIME,TIME_DELTA,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA,Zipcodes,Percent_Female,Percent_Non_White
0,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,10022,0.546,-85.4
1,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,11220,0.495,-29.6
2,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,11220,0.495,-29.6
3,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,10022,0.546,-85.4
4,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,10022,0.546,-85.4


In [48]:
mta_data_intra_w['Weighted_Intra'] = mta_data_intra_w['TOTAL_DELTA'].multiply(mta_data_intra_w['Percent_Female'],
                                                                          axis = 'index') 

In [49]:
mta_data_intra_w.head()

,C/A,UNIT,SCP,STATION,LINENAME,DATETIME,DATE,TIME,TIME_DELTA,ENTRIES_DELTA,EXITS_DELTA,TOTAL_DELTA,Zipcodes,Percent_Female,Percent_Non_White,Weighted_Intra
0,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,10022,0.546,-85.4,15.288
1,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,11220,0.495,-29.6,13.860
2,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,11220,0.495,-29.6,13.860
3,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,10022,0.546,-85.4,15.288
4,A002,R051,02-00-00,59 ST,NQR456W,2019-03-30,2019-03-30,00:00:00,04:00:00,20.0,8.0,28.0,10022,0.546,-85.4,15.288


In [50]:
with open('pickle/mta_data_intra_w.pickle', 'wb') as to_write:
    pickle.dump(mta_data_intra_w, to_write)
    
with open('pickle/mta_data_daily_w.pickle', 'wb') as to_write:
    pickle.dump(mta_data_daily_w, to_write)